In [23]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
# SETTINGS
learning_rate = 1e-2
torch.manual_seed(42)

device = 'cpu'
if torch.cuda.is_available():
	device = 'cuda'

print(device)

cpu


In [14]:
mnist_train = torchvision.datasets.MNIST('.', train=True, download=True)
mnist_test = torchvision.datasets.MNIST('.', train=False, download=True)

data_train = mnist_train.data.to(device).float()
data_test = mnist_test.data.to(device).float()

target_train = mnist_train.targets.to(device)
target_test = mnist_test.targets.to(device)

print(data_train.shape, data_test.shape)
print(data_train.dtype, data_test.dtype)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])
torch.float32 torch.float32


In [140]:
class Model(nn.Module):
	def __init__(self):
		super().__init__()
		self.net = nn.Sequential(
			nn.Linear(784, 800),
			nn.ReLU(),
			nn.Linear(800, 10),
			nn.Dropout(0.2),
		)
		
	def forward(self, x, y=None):
		r = x.view(-1, 784)
		r = self.net(r)
		# r = F.relu(self.linear1(r))
		# r = self.linear2(r)
		if y is None:
			loss = None
		else:
			loss = F.cross_entropy(r, y)
		return r, loss

model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [160]:
estimation_iters = 4
estimation_batch = 256

def get_split(split):
	data = data_train if split == 'train' else data_test
	target = target_train if split == 'train' else target_test
	return data, target

def get_batch(batch_size, split):
	data, target = get_split(split)
	ix = torch.randint(0, len(data), (batch_size,))
	x = torch.stack([data[i] for i in ix])
	y = torch.stack([target[i] for i in ix])
	return x, y

@torch.no_grad()
def estimate_loss(split = None):
	if split == None:
		return estimate_loss('train'), estimate_loss('test')
	losses = torch.zeros(estimation_iters)
	for i in range(estimation_iters):
		x, y = get_batch(estimation_batch, split)
		logits, loss = model.forward(x, y)
		losses[i] = loss
	return losses.mean().item()

@torch.no_grad()
def total_loss(split = None):
	if split == None:
		return total_loss('train'), total_loss('test')
	data, target = get_split(split)
	losses = torch.zeros(len(data) // estimation_batch)
	for i in range(len(data) // estimation_batch):
		lower = i * estimation_batch
		upper = min((i + 1) * estimation_batch, len(data))
		x = data[lower:upper]
		y = target[lower:upper]
		logits, loss = model.forward(x, y)
		losses[i] = loss
	return losses.mean().item()

In [157]:
training_iters = 100
training_batch = 64
eval_interval = 10
batch_losses = []
train_losses = []
test_losses = []
for iter in range(training_iters):
	if (iter % eval_interval) == 0:
		loss_train, loss_test = estimate_loss()
		test_losses.append(loss_test)
		train_losses.append(loss_train)
		print(f'iter {iter}: loss_train={loss_train}, loss_test={loss_test}')

	x, y = get_batch(training_batch, 'train')
	logits, loss = model(x, y)
	optimizer.zero_grad()
	loss.backward()
	optimizer.step()
	batch_losses.append(loss.item())

iter 0: loss_train=1.9520652294158936, loss_test=1.925053358078003
iter 10: loss_train=1.0562384128570557, loss_test=1.0887668132781982
iter 20: loss_train=1.3194245100021362, loss_test=1.2008187770843506
iter 30: loss_train=1.372653841972351, loss_test=1.0943793058395386
iter 40: loss_train=1.1309252977371216, loss_test=1.0781770944595337
iter 50: loss_train=1.1480441093444824, loss_test=0.91194748878479
iter 60: loss_train=0.926904022693634, loss_test=1.0020917654037476
iter 70: loss_train=1.1445530652999878, loss_test=1.0639041662216187
iter 80: loss_train=0.8845636248588562, loss_test=1.0733591318130493
iter 90: loss_train=0.9023007154464722, loss_test=0.9586383700370789


In [192]:
accuracy = 0
for i in range(len(data_test)):
	# result = model.predict(data_test[i])
	logits, _ = model(x)
	result = torch.multinomial(F.softmax(logits), 1)
	print(result.shape, result.dtype)
	if result == target_test[i]:
		accuracy += 1

accuracy /= len(data_test)
loss = total_loss('test')

torch.Size([64, 1]) torch.int64


C:\Users\v7\AppData\Local\Temp\ipykernel_20104\513220479.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  result = torch.multinomial(F.softmax(logits), 1)


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [188]:
import math

print(accuracy)
print(loss, math.exp(-loss))

0.7413
0.9198777675628662 0.3985677560153619


In [180]:
def predict_one(index):
	print(f'predict: {model.predict(data_test[index])}, target: {target_test[index]}')
	return mnist_test[index][0]

import random
predict_one(random.randint(0, data_test.shape[0]))

predict: 8, target: 9
